In [4]:
import pandas as pd 
import numpy as np
import os

In [7]:
root_dir = '/Users/liyao/Desktop/Tsuda_Lab/Source_code/AI-HDX-main/HDX_MS_dataset/AF_model/AF_cutter/'
df = pd.read_csv(root_dir + 'AFCT-OUT_summary.csv')
print(df.columns)
print(df.shape)

Index(['Unnamed: 0', 'PDB', 'chainID', 'type', 'start', 'end', 'len',
       'pLDDT_mean', 'seq'],
      dtype='object')
(87, 9)


In [8]:
filter_df = df[(df['pLDDT_mean'].astype(float) > 70)]
filter_df = filter_df[(filter_df['len'] > 50) & (filter_df['type'] == 'domain')]
print(filter_df.shape)

(40, 9)


In [ ]:
## rosseta relax ##
flag_relax = '/Users/liyao/Desktop/Tsuda_Lab/Source_code/rosetta_src_2021.16.61629_bundle/main/demos/tutorials/input_and_output/flag_input_relax'
os.environ['ROSETTA3'] = '/Users/liyao/Desktop/Tsuda_Lab/Source_code/rosetta_src_2021.16.61629_bundle/main/source/'
os.chdir(root_dir)

for index, row in filter_df.iterrows():
    len = row['len']
    pdb_id = row['PDB']
    chain = row['chainID']
    start = row['start']
    end = row['end']

    fname = f'{pdb_id}_AFCT-OUT_domain_{chain}{start}-{end}.pdb'
    finish_file = fname.replace('.pdb', '_0001.pdb')

    if os.path.exists(f'{root_dir}{finish_file}'):
        print(f'{root_dir}{finish_file} already exists!')
        continue
    elif not os.path.exists(f'{root_dir}{fname}'):
        print(f'{root_dir}{fname} does not exist!')
        continue
    print(fname)

    pdb_fpath = os.path.join(root_dir, fname)
    command = f'$ROSETTA3/bin/relax.default.macosclangrelease -in:file:s {pdb_fpath} @{flag_relax}'
    os.system(command)
    #break

In [ ]:
from output_dssp import output_dssp

PDB_file = '/Users/liyao/Desktop/Tsuda_Lab/Source_code/AI-HDX-main/complex_model/LptDEthanatin_complex_AF.pdb'
#output_dssp('LptDE-thanatin', PDB_file, '/Users/liyao/Desktop/Tsuda_Lab/Source_code/AI-HDX-main/complex_model')

lptDE_df = pd.read_excel('/Users/liyao/Desktop/Tsuda_Lab/Source_code/AI-HDX-main/dataset/LptD_Thanatin_LPS+Thanatin_State_SourceData_Fig3and4.xlsx')
print(lptDE_df.columns)
trim_LptDE_Apo = lptDE_df[(lptDE_df['Protein state'] == 'Apo') & (lptDE_df['HDX time (min)'] == 83.333)]
trim_LptDE_Thanatin = lptDE_df[(lptDE_df['Protein state'] == 'Thanatin') & (lptDE_df['HDX time (min)'] == 83.333)]
print(trim_LptDE_Apo.shape)
print(trim_LptDE_Thanatin.shape)

count = 0
for trim_df in [trim_LptDE_Apo, trim_LptDE_Thanatin]:
    count += 1
    if "MaxUptake" not in trim_df.columns:
        if 'Sequence' in trim_df.columns:
            trim_df['MaxUptake'] = trim_df['Sequence'].apply(lambda x: len(x) - 1 - x.count('P'))
            pass

    trim_df['MaxUptake'] = trim_df['MaxUptake'].astype(int)
    Uptake_columns = ['Uptake', 'Uptake (Da)', '#D']
    back_exchange_rate = 0.7

    for col in Uptake_columns:
        if col in trim_df.columns:
            trim_df['%D'] = trim_df[col] / (trim_df['MaxUptake'] * back_exchange_rate)
            over_range_mask = trim_df['%D'] > 1
            below_range_mask = trim_df['%D'] < 0
            trim_df.loc[over_range_mask, '%D'] = 1
            trim_df.loc[below_range_mask, '%D'] = 0
    if count == 1:
        fname = 'LptDE_Apo'
    elif count == 2:
        fname = 'LptDE_Thanatin'
    trim_df.to_csv(f'/Users/liyao/Desktop/Tsuda_Lab/Source_code/AI-HDX-main/complex_model/{fname}.csv', index=False)

In [14]:
from MSA_embedding import generate_embedding
import os

root_dir = '/Users/liyao/Desktop/Tsuda_Lab/Source_code/AI-HDX-main/complex_model/'
dssp_list = ['LptDE_Apo.dssp.txt', 'LptDE_Thanatin.dssp.txt']
hhm_file = 'LptD.hhm'

os.chdir(root_dir)
generate_embedding(hhm_file, dssp_list[0], 'LptDE_Apo.embedding.txt')
generate_embedding(hhm_file, dssp_list[1], 'LptDE_Thanatin.embedding.txt')

IndexError: list index out of range

In [ ]:
## run HBPLUS ##
import os
import shutil

hbplus_dir = '/Users/liyao/Desktop/Tsuda_Lab/Source_code/hbplus/'
PDB_dir = '/Users/liyao/Desktop/Tsuda_Lab/Source_code/AI-HDX-main/HDX_MS_dataset/AF_model/'
os.environ['PATH'] += os.pathsep + hbplus_dir
os.chdir(hbplus_dir)

for file in os.listdir(PDB_dir):
    target_file = f'{PDB_dir}AF_addH/{file.replace(".pdb", ".hb2")}'
    if os.path.isfile(target_file):
        print(f'{target_file} already exists!')
        continue
    if file.endswith('.pdb'):
        print(file)
        command = f'hbplus -O {PDB_dir}{file}'
        os.system(command)
        src_path = f'{hbplus_dir}{file.replace(".pdb", ".hb2")}'
        dst_path = f'{PDB_dir}AF_addH/{file.replace(".pdb", ".hb2")}'
        shutil.move(src_path, dst_path)
        src_path = f'{hbplus_dir}{file.replace(".pdb", ".h")}'
        dst_path = f'{PDB_dir}AF_addH/{file}'
        shutil.move(src_path, dst_path)

In [25]:
## run AF cutter ##

import pandas as pd
import os

root_dir = '/Users/liyao/Desktop/Tsuda_Lab/Source_code/AI-HDX-main/HDX_MS_dataset/AF_model/AF_cutter/'
df = pd.read_csv(root_dir + 'AFCT-OUT_summary.csv')

filter_df = df[(df['pLDDT_mean'].astype(float) > 70)]
filter_df = filter_df[(filter_df['len'] > 50) & (filter_df['type'] == 'domain')]
print(filter_df.shape)

(42, 9)


In [ ]:
## run FIRST rigidity ##
from RUN_FIRST import RUN_FIRST, copyfiles
import os
import shutil

proflex_dir = '/Users/liyao/Desktop/Tsuda_Lab/Source_code/ProFlex-master/proflex/'
PDB_dir = '/Users/liyao/Desktop/Tsuda_Lab/Source_code/AI-HDX-main/HDX_MS_dataset/AF_model/AF_addH/'
dst_dir = '/Users/liyao/Desktop/Tsuda_Lab/Source_code/AI-HDX-main/HDX_MS_dataset/AF_model/AF_proflex/'
count = 0

for file in os.listdir(PDB_dir):
    if file.endswith('.pdb'):
        print(file)
        file = file.replace('.pdb', '')

        if not os.path.isfile(f'{dst_dir}decomp_{file}'):
            with open(f'{PDB_dir}{file}.pdb', 'r') as f, open(f'{proflex_dir}{file}.pdb', 'w') as w:
                for i in f:
                    if i.startswith('ATOM'):
                        w.write(i)
            try:
                RUN_FIRST(f'{file}.pdb', '-h')
                copyfiles(file, proflex_dir, dst_dir)
                count += 1
            except:
                print(f'{file} failed!')
                continue
        #break

print(count)

In [ ]:
## output residue dssp value ##
from prot_rigidity import DSSP_calculation

root_dir = '/Users/liyao/Desktop/Tsuda_Lab/Source_code/AI-HDX-main/HDX_MS_dataset/AF_model'

DSSP_calculation(f'{root_dir}/AF_addH', f'{root_dir}/AF_dssp')

In [15]:
## output residue ANSSURR rigidity ##
from prot_rigidity import ASSURR_flexibility

root_dir = '/Users/liyao/Desktop/Tsuda_Lab/Source_code/AI-HDX-main/HDX_MS_dataset/AF_model'

ASSURR_flexibility(f'{root_dir}/AF_proflex', f'{root_dir}/AF_dssp', f'{root_dir}/AF_rigidity')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/liyao/Desktop/Tsuda_Lab/Source_code/AI-HDX-main/HDX_MS_dataset/AF_model/AF_dssp/P62873_AF/dssp_P62873_AF.txt'